# Model training notebook
>This notebook uses Tensorflow Object Detection API to train the model.

## 1. Prepare the environment

### Import every dependency used
>Restart runtime after this command

In [1]:
!python -m pip install numpy==1.17.4

     |████████████████████████████████| 20.0MB 1.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.4
    Uninstalling numpy-1.18.4:
      Successfully uninstalled numpy-1.18.4


In [1]:
%tensorflow_version 1.x

import tensorflow as tf
import os
import shutil
import glob
import urllib.request
import tarfile
import zipfile
import shutil
import re
import numpy as np
tf.__version__
np.__version__

TensorFlow 1.x selected.


'1.17.4'

In [2]:
!nvidia-smi

Thu May 21 11:17:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Define paths

>Paths defined to work remotely on google drive.

In [0]:
rootPath = '/content/drive/My Drive/Machine Learning/License plate detection'
dataPath = rootPath + '/data'
savedPath = rootPath + '/saved'
modelsPath = savedPath + '/models'

localPath = '/content'
localDataPath = localPath + '/data'
modelDir = localPath + '/training'

### Mount Google Drive to this Notebook instance
>As the dataset has been prepared previously and updated to Google Drive, the model building and training process will be done there, not locally.

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

os.chdir(localPath)
# Show current directory
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content


### Prepare and test Object Detection module

In [5]:
%cd /content

!git clone --quiet https://github.com/tensorflow/models.git
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

%cd /content
repo_url = 'https://github.com/Tony607/object_detection_demo'
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
Selecting previously unselected package python-bs4.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpa

## 2. Configuration

### Choose a pre-trained model

>The model used for this project is `ssd_mobilenet_v3_large_coco`.
Other models are available [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models).
>Real time inference is necessary, so a model with good inference speed (`ms`) needed to be chosen with a relativly high `mAP` on COCO.

In [0]:
# Number of training steps
num_steps = 40000

# Number of evaluation steps
num_eval_steps = 1000

batch_size = 32

In [0]:
# Models to work with
MODELS_CONFIG = {

    'ssd_quantized_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03',
        'pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'batch_size': 32,
    },

    'ssdlite_mobilenet_v2': {
        'model_name': 'ssdlite_mobilenet_v2_coco_2018_05_09',
        'pipeline_file': 'ssdlite_mobilenet_v2_coco.config',
        'batch_size': 32,
    },

    'ssd_mobilenet_v2_oid': {
        'model_name': 'ssd_mobilenet_v2_oid_v4_2018_12_12',
        'pipeline_file': 'ssd_mobilenet_v2_oid_v4.config',
        'batch_size': 32,
    },

    'ssd_mobilenet_v3_small_coco': {
        'model_name': 'ssd_mobilenet_v3_small_coco_2019_08_14',
        'pipeline_file': 'ssdlite_mobilenet_v3_small_320x320_coco.config',
        'batch_size': 32,
    }

}

# Select a model in `MODELS_CONFIG`.
selectedModel = 'ssd_mobilenet_v3_small_coco'

In [0]:
# Name of the object detection model to use
MODEL = MODELS_CONFIG[selectedModel]['model_name']

# Name of the pipline file in tensorflow object detection API
pipeline_file = MODELS_CONFIG[selectedModel]['pipeline_file']

# Training batch size fits in Colab's Tesla K80 GPU memory for selected model
batch_size = MODELS_CONFIG[selectedModel]['batch_size']

## 3. Optional: download base model
>Download the selected model in the MODELS_CONFIG file

In [0]:
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

### Check downloaded model content
>This is the directory of the "fine_tune_checkpoint" that is used in the config file

In [10]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 35M
drwxr-xr-x  2 141711 89939 4.0K Oct 14  2019 .
drwxr-xr-x 63 root   root  4.0K May 21 11:19 ..
-rw-r--r--  1 141711 89939   77 Oct 14  2019 checkpoint
-rw-r--r--  1 141711 89939 7.2M Oct 14  2019 frozen_inference_graph.pb
-rw-r--r--  1 141711 89939  14M Oct 14  2019 model.ckpt.data-00000-of-00001
-rw-r--r--  1 141711 89939  11K Oct 14  2019 model.ckpt.index
-rw-r--r--  1 141711 89939 6.6M Oct 14  2019 model.ckpt.meta
-rw-r--r--  1 141711 89939 6.8M Oct 14  2019 model.tflite
-rw-r--r--  1 141711 89939 4.7K Oct 14  2019 pipeline.config


In [11]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

In [0]:
pipelineFilePath = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)
assert os.path.isfile(pipelineFilePath), '`{}` not exist'.format(pipelineFilePath)

## 4. Optional: import saved model
>If a previously saved model is selected to work with. The `zip` file is placed on Google Drive, but working there would be slow so needs to be extracted locally.

In [0]:
# Name of the object detection model to load
modelToLoad = 'ssd_mobilenet_v3_small_coco_trained_at_2020-05-17_adam_small_lr_10k_training'

modelDir = localPath + '/' + 'training'
pipelineFilePath = modelDir + '/' + modelToLoad + '/' + 'pipeline.config'

os.chdir(localPath)

if (os.path.exists(modelDir)):
  shutil.rmtree(modelDir)

os.mkdir(modelDir)
os.chdir(modelDir)

# Load the frozen model that is needed for inference
# Run it only when the model is not yet extracted 
zipRef = zipfile.ZipFile(modelsPath + '/' + modelToLoad + '.zip', 'r')
zipRef.extractall(modelDir + '/')
zipRef.close()
modelDir = localPath + '/' + 'training' + '/' + modelToLoad
print('Model extraction successful')

Model extraction successful


## 5. Dataset preparation

### Extract `zip` dataset locally
>To access the dataset. The `zip` file is placed on Google Drive, but working there would be slow so needs to be extracted locally.

In [13]:
# Run it only when the dataset is not yet extracted 
zipRef = zipfile.ZipFile(dataPath + "/datasetRecords.zip", 'r')
zipRef.extractall(localDataPath)
zipRef.close()
print('Dataset extraction successful')

Dataset extraction successful


In [0]:
train_record_fname = '/content/data/trainDataset.tfrecord-?????-of-00010'
validation_record_fname = '/content/data/validationDataset.tfrecord-?????-of-00001'
label_map_pbtxt_fname = '/content/data/classes.pbtxt'

## 6. Modify configuration file

In [0]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

####Optional: Edit configuration file
>Change config file based on selected model samples [here](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).
>
>Add path to `tfrecod` files and to the `txt` file. Teaching parameters can be selected here (class number, batch size). Hyperparameters (augmentation, batch dropout, batch normalization, etc.) can be tuned here too.

In [16]:
print(pipelineFilePath)

/content/models/research/object_detection/samples/configs/ssdlite_mobilenet_v3_small_320x320_coco.config


In [29]:
%%writefile {pipelineFilePath}

# SSDLite with Mobilenet v3 small feature extractor.
# Trained on COCO14, initialized from scratch.
# TPU-compatible.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    #change num_classes
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.8
        kernel_size: 3
        use_depthwise: true
        box_code_size: 4
        apply_sigmoid_to_scores: false
        class_prediction_bias_init: -4.6
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            random_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.97,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v3_small'
      min_depth: 16
      depth_multiplier: 1.0
      use_depthwise: true
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.97,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
       num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    normalize_loc_loss_by_codesize: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: true
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  #change batch_size
  batch_size: 32
  sync_replicas: true
  startup_delay_steps: 0
  replicas_to_aggregate: 32
  #change num_steps
  num_steps: 40000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }
  optimizer {
    adam_optimizer: {
      learning_rate: {
        constant_learning_rate {
          learning_rate: 0.00004
        }
      #  cosine_decay_learning_rate {
      #    learning_rate_base: 0.00004
      #    #change total_steps
      #    total_steps: 40000
      #    warmup_learning_rate: 0.00001
      #    #change warmup_steps to include 1-2 epoch(s)
      #    warmup_steps: 1500
      #  }
      }
    }
    use_moving_average: false
  }
  #Use it when you use a starter model
  fine_tune_checkpoint: "/content/models/research/pretrained_model/model.ckpt"
  #Use it when you continue your model's training
  #fine_tune_checkpoint: "/content/training/model.ckpt-#####"
  from_detection_checkpoint: true
  load_all_detection_checkpoint_vars: true
  #change max_number_of_boxes
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  freeze_variables: ".*FeatureExtractor*."
}

train_input_reader: {
  tf_record_input_reader {
    #change input_path
    input_path: "/content/data/trainDataset.tfrecord-?????-of-00010"
  }
  #change label_map_path
  label_map_path: "/content/data/classes.pbtxt"
}

eval_config: {
  #change num_examples
  num_examples: 1000
}

eval_input_reader: {
  tf_record_input_reader {
    #change input_path
    input_path: "/content/data/validationDataset.tfrecord-?????-of-00001"
  }
  #change label_map_path
  label_map_path: "/content/data/classes.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting /content/training/pipeline.config


In [30]:
!cat {pipelineFilePath}


# SSDLite with Mobilenet v3 small feature extractor.
# Trained on COCO14, initialized from scratch.
# TPU-compatible.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    #change num_classes
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode

### Optional: remove previous local model output directory to fresh start

In [0]:
# Optionally remove content in output model directory to fresh start.
!rm -rf {modelDir}

## 7. Run Tensorboard

In [20]:
os.chdir(localPath)
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-05-21 11:27:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.54.253.53, 18.208.90.15, 3.220.228.61, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.54.253.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  18.1MB/s    in 0.7s    

2020-05-21 11:27:33 (18.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = modelDir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

### Get Tensorboard link
>Tensorboard magic: re-run field if an error occured

In [23]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://62b3f178.ngrok.io


## 8. Train the model

### Move pipeline file to the train folder

In [24]:
os.makedirs(modelDir, exist_ok=True)
shutil.copyfile(pipelineFilePath, modelDir + '/pipeline.config')
pipelineFilePath = modelDir + '/pipeline.config'
print(pipelineFilePath)

/content/training/pipeline.config


### Start training

In [31]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipelineFilePath} \
    --model_dir={modelDir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}



W0521 11:34:29.262147 140130932348800 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:137: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0521 11:34:29.265403 140130932348800 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:685: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0521 11:34:29.265526 140130932348800 model_lib.py:686] Forced number of epochs for all eval validations to be 1.

W0521 11:34:29.265608 140130932348800 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:523: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Maybe overwriting train_steps: 40000
I0521 11:34:29.265668 140130932348800 config_util.py:523] Maybe overwriting train_steps: 40000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0521 11:34:29.265725 140130

### Check output directory after training

In [33]:
!ls {modelDir}

checkpoint
eval_0
events.out.tfevents.1590060883.a492ad8563e5
export
graph.pbtxt
model.ckpt-35809.data-00000-of-00001
model.ckpt-35809.index
model.ckpt-35809.meta
model.ckpt-37049.data-00000-of-00001
model.ckpt-37049.index
model.ckpt-37049.meta
model.ckpt-38230.data-00000-of-00001
model.ckpt-38230.index
model.ckpt-38230.meta
model.ckpt-39546.data-00000-of-00001
model.ckpt-39546.index
model.ckpt-39546.meta
model.ckpt-40000.data-00000-of-00001
model.ckpt-40000.index
model.ckpt-40000.meta
pipeline.config


## 9. Export model
>Save the model to Drive as a `zip` file.

In [34]:
os.chdir(localPath)
# Location where the exported model will be saved
exportName = selectedModel + '_trained_at_' + '2020-05-21_adam_00004_const_lr'
print(exportName)

ssd_mobilenet_v3_small_coco_trained_at_2020-05-21_adam_00004_const_lr


### Optional: Zip all the training files to continue training later

In [35]:
exportToDrivePath = modelsPath + '/' + exportName
outputTrainingFileName = localPath + '/' + exportName + '_training'
os.makedirs(localPath + '/' + 'full_training')
shutil.copytree(localPath + '/training', localPath + '/' + 'full_training' + '/' + exportName + '_training/')

'/content/full_training/ssd_mobilenet_v3_small_coco_trained_at_2020-05-21_adam_00004_const_lr_training/'

In [36]:
#ignore the error if zip creation is successful
shutil.make_archive(outputTrainingFileName, 'zip', localPath + '/' + 'full_training')

'/content/ssd_mobilenet_v3_small_coco_trained_at_2020-05-21_adam_00004_const_lr_training.zip'

In [37]:
shutil.move(outputTrainingFileName + '.zip', modelsPath)

'/content/drive/My Drive/Machine Learning/License plate detection/saved/models/ssd_mobilenet_v3_small_coco_trained_at_2020-05-21_adam_00004_const_lr_training.zip'

### Optional: convert and save only the model to the local root folder

In [0]:
output_directory = '/content/fine_tuned_model/' + exportName

lst = os.listdir(modelDir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(modelDir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipelineFilePath} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

/content/training/model.ckpt-30000


W0517 18:54:38.041493 140700720269184 module_wrapper.py:139] From /content/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0517 18:54:38.049091 140700720269184 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:419: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0517 18:54:38.049462 140700720269184 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:138: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0517 18:54:38.088702 140700720269184 module_wrapper.py:139] From /content/models/research/object_detection/core/preprocessor.py:3030: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.


W0517 18:54:38.121094 140700720269184 module_wrapper.py:139] From /content/models/research/object_detect

#### Zip the model

In [0]:
exportToDrivePath = modelsPath + '/' + exportName
outputFileName = localPath + '/' + exportName

#ignore the error if zip creation is successful
shutil.make_archive(outputFileName, 'zip', localPath + '/fine_tuned_model')

'/content/ssd_mobilenet_v3_small_coco_trained_at_2020-05-17_momentum.zip'

#### Move the `zip`ped file to the Drive folder

In [0]:
shutil.move(outputFileName + '.zip', modelsPath)

'/content/drive/My Drive/Machine Learning/License plate detection/saved/models/ssd_mobilenet_v3_small_coco_trained_at_2020-05-17_momentum.zip'